In [1]:
import re
import MeCab
import pickle
import numpy as np
import data_helpers as dh
import pandas as pd
import time
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.manifold import MDS,TSNE
from collections import Counter
from collections import OrderedDict
from gensim.models import word2vec
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
All_df = pd.read_pickle("../data/all_mecab.pickle")
All_df.head(3)

# senとsen_preの単語をIDに変換し、新たな列としてAll_dfに追加する

,sen,sen_pre,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,"[よろしく, お願い, し, ます, ！, EOS]",[EOS],1,-1,1234568,まこぴす,よろしくお願いします！,5,5,1,1,1,1,0,0,0,0
1,"[よろしく, お願い, し, ます, EOS]","[よろしく, お願い, し, ます, ！, EOS]",31,-1,1234568,哲,よろしくお願いします,5,5,1,1,1,1,0,0,0,0
2,"[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...","[よろしく, お願い, し, ます, EOS]",70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,1,1,1,1,0,0,0,0


# senとsen_preの単語をIDに変換し、新たな列としてAll_dfに追加する

In [3]:
sen = All_df['sen'].values
sen_pre = All_df['sen_pre'].values

# 単語辞書の作成
wd_set = Counter([x for s in (sen + sen_pre) for x in s])
wd_ary = np.array(list(wd_set.keys()))
wd_cnt = np.array(list(wd_set.values()))

# 出現頻度順にソート
wd_ary = wd_ary[np.argsort(wd_cnt)[::-1]]
wd_cnt.sort()
wd_cnt = wd_cnt[::-1]

# 単語ID辞書の作成
wd_to_id = {wd: i for i, wd in enumerate(wd_ary)}

# Top10の単語を出力
for i in range(10):
    print("単語:",
          list(wd_ary)[i], "\t出現数:",
          list(wd_cnt)[i], "\tID:", wd_to_id[list(wd_ary)[i]])

# 出現数CUT_OFF以下の単語のIDを統一
CUT_OFF = 2
print("words kinds:", len(wd_cnt), "words>=" + str(CUT_OFF) + ":",
      np.sum(wd_cnt >= CUT_OFF))
other_id = np.sum(wd_cnt >= CUT_OFF)
wd_to_id.update({wd: other_id for wd in wd_ary[wd_cnt < CUT_OFF]})
id_to_wd = {wd_to_id[wd]: wd for wd in wd_to_id.keys()}

# senとsen_preの単語をIDに変換
sen_id = []
sen_pre_id = []
for s, s_pre in zip(sen, sen_pre):
    sen_id.append([str(wd_to_id[wd]) for wd in s])
    sen_pre_id.append([str(wd_to_id[wd]) for wd in s_pre])

# 新し列としてAll_dfに追加
All_df.insert(loc=0, column='sen_id', value=sen_id)
All_df.insert(loc=1, column='sen_pre_id', value=sen_pre_id)

単語: EOS 	出現数: 29580 	ID: 0
単語: の 	出現数: 10567 	ID: 1
単語: て 	出現数: 7408 	ID: 2
単語: です 	出現数: 7390 	ID: 3
単語: ます 	出現数: 7363 	ID: 4
単語: か 	出現数: 7285 	ID: 5
単語: 、 	出現数: 6959 	ID: 6
単語: に 	出現数: 6914 	ID: 7
単語: が 	出現数: 6893 	ID: 8
単語: は 	出現数: 6793 	ID: 9
words kinds: 6961 words>=2: 6649


In [4]:
All_df.head(3)

,sen_id,sen_pre_id,sen,sen_pre,say_id,reply_id,group_id,name,body,16types_a,16types_b,argument_a,argument_b,epistemic_a,epistemic_b,social_a,social_b,coordination_a,coordination_b
0,"[39, 35, 12, 4, 18, 0]",[0],"[よろしく, お願い, し, ます, ！, EOS]",[EOS],1,-1,1234568,まこぴす,よろしくお願いします！,5,5,1,1,1,1,0,0,0,0
1,"[39, 35, 12, 4, 0]","[39, 35, 12, 4, 18, 0]","[よろしく, お願い, し, ます, EOS]","[よろしく, お願い, し, ます, ！, EOS]",31,-1,1234568,哲,よろしくお願いします,5,5,1,1,1,1,0,0,0,0
2,"[413, 19, 1, 8, 2309, 3, 0, 39, 35, 12, 4, 18, 0]","[39, 35, 12, 4, 0]","[名前, な, の, が, 恥ずかしい, です, EOS, よろしく, お願い, し, ます...","[よろしく, お願い, し, ます, EOS]",70,-1,1234568,仙波,名前なのが恥ずかしいです…\nよろしくお願いします！,5,5,1,1,1,1,0,0,0,0


# word2vec を利用し、単語のベクトル辞書を作成する

In [5]:
sen_id = All_df['sen_id'].values
sen_pre_id = All_df['sen_pre_id'].values
sen_all = np.hstack((sen_id, sen_pre_id))

max_sen_length = max([len(sen) for sen in sen_all])
print("max_sen_length:", max_sen_length)

word_vectors_size = 200

model = dh.get_w2v_model()

max_sen_length: 292


# （重要）各センテンスの長さを66に統一する

In [6]:
All_df['sen_id'] = [x[:66] for x in All_df['sen_id']]
All_df['sen_pre_id'] = [x[:66] for x in All_df['sen_pre_id']]

sen_all = np.hstack((All_df['sen_id'].values, All_df['sen_pre_id'].values))
max_sen_length = max([len(sen) for sen in sen_all])
print("max_sen_length:", max_sen_length)

max_sen_length: 66


# データの準備
* sentences dataをpaddingし、word vectorsによりfeature vectorsを作る
* labels dataをone hotの型に変換する

In [7]:
# データの整理（一致、重複）
print("全データ(All_df)：", All_df.shape)
All_drop_df = All_df.drop_duplicates(subset=['body', 'name']).reset_index(drop=True)
print("重複投稿を排除したデータ(All_drop_df)：", All_drop_df.shape, "\n")



all_sen = All_drop_df['sen_id'].values
all_sen = np.array([np.array(x, dtype=np.int32) for x in all_sen])
x = dh.sen_to_fv(all_sen, max_sen_length, model, False)
print("input data(sen)：",x.shape)
all_sen_pre = All_drop_df['sen_pre_id'].values
all_sen_pre = np.array([np.array(x, dtype=np.int32) for x in all_sen_pre])
x_pre = dh.sen_to_fv(all_sen_pre, max_sen_length, model, False)
print("input data(sen_pre)：",x_pre.shape)


全データ(All_df)： (12012, 19)
重複投稿を排除したデータ(All_drop_df)： (11357, 19) 

input data(sen)： (11357, 66, 200)
input data(sen_pre)： (11357, 66, 200)


In [8]:
# 16types-------------------------------------
print("16types_a：", Counter(All_df['16types_a']), "\n")

label_t16_a = All_drop_df['16types_a'].values
label_t16_a = np.array(label_t16_a, dtype=np.int32)
y_t16_a = dh.labels_to_one_hot(label_t16_a, 16)
print("input data(t16_a)：", y_t16_a.shape)

label_arg_a = All_drop_df['argument_a'].values
label_arg_a = np.array(label_arg_a, dtype=np.int32)
y_arg_a = dh.labels_to_one_hot(label_arg_a, 6)
print("input data(arg_a)：", y_arg_a.shape)

label_epi_a = All_drop_df['epistemic_a'].values
label_epi_a = np.array(label_epi_a, dtype=np.int32)
y_epi_a = dh.labels_to_one_hot(label_epi_a, 4)
print("input data(epi_a)：", y_epi_a.shape)


data = dh.set_data_sets_3(x, x_pre, y_t16_a, y_arg_a, y_epi_a)


print(data.test.labels_1.shape)
print(data.test.labels_2.shape)
print(data.test.labels_3.shape)


16types_a： Counter({1: 2425, 2: 1938, 6: 1307, 3: 1301, 4: 1230, 5: 1224, 8: 550, 9: 510, 7: 393, 14: 310, 15: 218, 11: 184, 0: 143, 10: 123, 12: 100, 13: 56}) 

input data(t16_a)： (11357, 16)
input data(arg_a)： (11357, 6)
input data(epi_a)： (11357, 4)
(1136, 16)
(1136, 6)
(1136, 4)


In [9]:
embedding_dim = word_vectors_size
num_classes_1 = 16
num_classes_2 = 6
num_classes_3 = 4
sequence_length = max_sen_length
num_steps = max_sen_length
num_lstm_hidden = 200
num_hidden = 200

train_dropout = 0.8
test_dropout = 1.0

learning_rate = 0.001
batch_size = 64
total_batch = int(data.train.num_examples / batch_size)
print("total_batch:", total_batch)
training_epochs = 40

# tf Graph input
X = tf.placeholder(dtype=tf.float32, shape=[None, sequence_length, embedding_dim])
X_pre = tf.placeholder(dtype=tf.float32, shape=[None, sequence_length, embedding_dim])
Y_1 = tf.placeholder(dtype=tf.float32, shape=[None, num_classes_1])
Y_2 = tf.placeholder(dtype=tf.float32, shape=[None, num_classes_2])
Y_3 = tf.placeholder(dtype=tf.float32, shape=[None, num_classes_3])
keep_prob = tf.placeholder(tf.float32)

# Store layers weight & bias
weights = {
    'fc1':tf.Variable(tf.truncated_normal(stddev=0.1, shape=[2*num_lstm_hidden, num_hidden])),
    'out_1':tf.Variable(tf.truncated_normal(stddev=0.1, shape=[num_hidden, num_classes_1])),
    'out_2':tf.Variable(tf.truncated_normal(stddev=0.1, shape=[num_hidden, num_classes_2])),
    'out_3':tf.Variable(tf.truncated_normal(stddev=0.1, shape=[num_hidden, num_classes_3]))
}
biases = {
    'fc1': tf.Variable(tf.constant(value=0.1, shape=[num_hidden])),
    'out_1': tf.Variable(tf.constant(value=0.1, shape=[num_classes_1])),
    'out_2': tf.Variable(tf.constant(value=0.1, shape=[num_classes_2])),
    'out_3': tf.Variable(tf.constant(value=0.1, shape=[num_classes_3]))
}


total_batch: 159


In [10]:
def Si_S2S(x, x_pre, weights, biases, dropout):

    num_layers = 2
    
    with tf.variable_scope('encoder'):
        enc_cell = tf.nn.rnn_cell.BasicLSTMCell(num_lstm_hidden)
        multi_enc_cell = tf.nn.rnn_cell.MultiRNNCell([enc_cell] * num_layers)
        enc_outputs, enc_states = tf.nn.dynamic_rnn(multi_enc_cell, x, dtype=tf.float32)
        enc_output = tf.concat(enc_outputs, axis=2)
    with tf.variable_scope('decoder'):
        dec_cell = tf.nn.rnn_cell.BasicLSTMCell(num_lstm_hidden)
        multi_dec_cell = tf.nn.rnn_cell.MultiRNNCell([dec_cell] * num_layers)
        dec_outputs, dec_states = tf.nn.dynamic_rnn(multi_dec_cell, x_pre, initial_state=enc_states)
        dec_output = tf.concat(dec_outputs, axis=2)
    
    

    # Full connection layer
    enc_out = tf.reduce_mean(enc_output, axis=1)
    dec_out = tf.reduce_mean(dec_output, axis=1)
    s2s_out = tf.concat([enc_out, dec_out], axis=1)
    
    fc1 = tf.add(tf.matmul(s2s_out, weights['fc1']), biases['fc1'])
    fc1_relu = tf.nn.relu(fc1)
    fc1_drop = tf.nn.dropout(fc1_relu, dropout)
    
    out_1 = tf.add(tf.matmul(fc1_drop, weights['out_1']), biases['out_1'])
    out_2 = tf.add(tf.matmul(fc1_drop, weights['out_2']), biases['out_2'])
    out_3 = tf.add(tf.matmul(fc1_drop, weights['out_3']), biases['out_3'])
    return out_1, out_2, out_3

In [11]:
# Construct model
y_pred_1, y_pred_2, y_pred_3 = Si_S2S(X, X_pre, weights, biases, keep_prob)

# y_softmax = tf.nn.softmax(y_pred)

# Define loss and optimizer
# type 1(old):
# loss = tf.reduce_mean(
#     -tf.reduce_sum(Y * tf.log(y_softmax), reduction_indices=[1]))
# type 2(server):
# loss = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_pred))
# type 3(new):
# loss = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=y_pred))


loss_1 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y_1, logits=y_pred_1))

loss_2 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y_2, logits=y_pred_2))

loss_3 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y_3, logits=y_pred_3))


joint_loss = loss_1 + loss_2 + loss_3

optimizer_1 = tf.train.AdamOptimizer(learning_rate).minimize(loss_1)
optimizer_2 = tf.train.AdamOptimizer(learning_rate).minimize(loss_2)
optimizer_3 = tf.train.AdamOptimizer(learning_rate).minimize(loss_3)

optimizer_joint = tf.train.AdamOptimizer(learning_rate).minimize(joint_loss)

# Evaluate model
pred_1 = tf.argmax(y_pred_1, 1)
true_1 = tf.argmax(Y_1, 1)
correct_prediction_1 = tf.equal(pred_1, true_1)
accuracy_1 = tf.reduce_mean(tf.cast(correct_prediction_1, tf.float32))


pred_2 = tf.argmax(y_pred_2, 1)
true_2 = tf.argmax(Y_2, 1)
correct_prediction_2 = tf.equal(pred_2, true_2)
accuracy_2 = tf.reduce_mean(tf.cast(correct_prediction_2, tf.float32))

pred_3 = tf.argmax(y_pred_3, 1)
true_3 = tf.argmax(Y_3, 1)
correct_prediction_3 = tf.equal(pred_3, true_3)
accuracy_3 = tf.reduce_mean(tf.cast(correct_prediction_3, tf.float32))



In [12]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Training cycle
all_test_x = data.test.vectors_1
all_test_x_pre = data.test.vectors_2
all_test_y_1 = data.test.labels_1
all_test_y_2 = data.test.labels_2
all_test_y_3 = data.test.labels_3

start = time.time()
for epoch_i in range(training_epochs):
    ave_cost = 0
    for batch_i in range(total_batch):
        batch_x, batch_x_pre, batch_y_1, batch_y_2, batch_y_3 = data.train.next_batch(batch_size, 3)
        _, c = sess.run(
            [optimizer_joint, joint_loss],
            feed_dict={
                X: batch_x,
                X_pre: batch_x_pre,
                Y_1: batch_y_1,
                Y_2: batch_y_2,
                Y_3: batch_y_3,
                keep_prob: train_dropout
            })
        ave_cost += c / total_batch
    # Display logs per epoch step
    if epoch_i % 1 == 0:
        train_acc_1 = sess.run(
            accuracy_1,
            feed_dict={
                X: batch_x,
                X_pre: batch_x_pre,
                Y_1: batch_y_1,
                keep_prob: test_dropout
            })
        train_acc_2 = sess.run(
            accuracy_2,
            feed_dict={
                X: batch_x,
                X_pre: batch_x_pre,
                Y_2: batch_y_2,
                keep_prob: test_dropout
            })
        train_acc_3 = sess.run(
            accuracy_3,
            feed_dict={
                X: batch_x,
                X_pre: batch_x_pre,
                Y_3: batch_y_3,
                keep_prob: test_dropout
            })
        
        test_acc_1 = sess.run(
            accuracy_1,
            feed_dict={
                X: all_test_x,
                X_pre: all_test_x_pre,
                Y_1: all_test_y_1,
                keep_prob: test_dropout
            })
        test_acc_2 = sess.run(
            accuracy_2,
            feed_dict={
                X: all_test_x,
                X_pre: all_test_x_pre,
                Y_2: all_test_y_2,
                keep_prob: test_dropout
            })
        test_acc_3 = sess.run(
            accuracy_3,
            feed_dict={
                X: all_test_x,
                X_pre: all_test_x_pre,
                Y_3: all_test_y_3,
                keep_prob: test_dropout
            })
        print("Ep:%3d Bc:%4d" % (epoch_i + 1, batch_i + 1),
              "| train_1=%.3f" % train_acc_1, "test_1=%.3f" % test_acc_1, 
              "| train_2=%.3f" % train_acc_2, "test_2=%.3f" % test_acc_2, 
              "| train_3=%.3f" % train_acc_3, "test_3=%.3f" % test_acc_3, "| joint_loss=%5.3f" % ave_cost,)
end = time.time()
print("Process Time :%.2f s" % (end - start))
sess.close()

Ep:  1 Bc: 159 | train_1=0.234 test_1=0.164 | train_2=1.000 test_2=0.194 | train_3=1.000 test_3=0.180 | joint_loss=4.278
Ep:  2 Bc: 159 | train_1=0.453 test_1=0.389 | train_2=0.734 test_2=0.590 | train_3=0.766 test_3=0.642 | joint_loss=4.183
Ep:  3 Bc: 159 | train_1=0.453 test_1=0.447 | train_2=0.672 test_2=0.606 | train_3=0.734 test_3=0.661 | joint_loss=3.605
Ep:  4 Bc: 159 | train_1=0.406 test_1=0.472 | train_2=0.562 test_2=0.599 | train_3=0.594 test_3=0.666 | joint_loss=3.416
Ep:  5 Bc: 159 | train_1=0.531 test_1=0.489 | train_2=0.734 test_2=0.624 | train_3=0.766 test_3=0.667 | joint_loss=3.285
Ep:  6 Bc: 159 | train_1=0.516 test_1=0.493 | train_2=0.703 test_2=0.629 | train_3=0.750 test_3=0.672 | joint_loss=3.219
Ep:  7 Bc: 159 | train_1=0.406 test_1=0.519 | train_2=0.688 test_2=0.640 | train_3=0.719 test_3=0.679 | joint_loss=3.132
Ep:  8 Bc: 159 | train_1=0.516 test_1=0.522 | train_2=0.656 test_2=0.635 | train_3=0.672 test_3=0.671 | joint_loss=3.043
Ep:  9 Bc: 159 | train_1=0.531 t